# 事例6：最適化として解く主成分分析（PCA）― PyTorch実装と固有値分解PCAとの同値性検証 ―

## 目的

本事例では、主成分分析（PCA）を最適化問題として定式化し、勾配法により実装することで、固有値分解による線形代数的定義と同一の主成分・主成分得点・寄与率が得られることを確認する。

## 目次

1. 位置づけ（事例5との関係）
2. データ仕様

   2.1 生成モデル

   2.2 パラメータ設定

3. 手法

   3.1 中心化

   3.2 射影と目的関数

   3.3 勾配最適化と正規化制約

3. 出力

4. 検証

5. スコープ外

## 1. 位置づけ（事例5との関係）

本事例では、事例5で固有値分解により求めたPCAを、最適化問題として直接解く。
分散・共分散行列を明示的に構成せず、勾配法により主成分負荷量を求め、両手法が同一解に到達することを検証する。

## 2. データ仕様

本事例で用いる合成データについて、生成モデルおよびパラメータ設定を示す。

### 2.1 生成モデル

本事例では、潜在因子に基づく線形生成モデルにより合成データを作成する。
まず、平均0・分散1の正規分布に従う潜在因子行列
$Z \in \mathbb{R}^{n \times k}$
を生成する。
次に、混合行列
$W \in \mathbb{R}^{k \times d}$
によって潜在因子を観測空間へ線形変換し、ノイズ行列
$E \in \mathbb{R}^{n \times d}$
を加えることで、観測データ
$X \in \mathbb{R}^{n \times d}$
を次式で定義する。

$$
X = ZW + E
$$

### 2.2 パラメータ設定

本事例で用いるパラメータ設定を以下に示す。
- サンプル数：$n = 500$
- 観測次元：$d = 5$
- 潜在因子数：$k = 2$
- ノイズ強度：$\sigma = 0.3$
- 乱数シード：$42$

## 3. 手法

本事例では、主成分分析（PCA）を最適化問題として定式化し、勾配法に基づく数値計算手順として実装する。
可視化・解釈・既存ライブラリとの比較は行わず、中心化 → 射影 → 最適化、の各操作が数式どおり PyTorch 上で対応していることのみを目的とする。

以下では、各ステップを数式と実装単位で対応づけて記述する。

### 3.1 中心化

$$
X \in \mathbb{R}^{n \times d}
$$

$$
x_i \in \mathbb{R}^d \quad (i=1,\dots,n)
$$

$$
\mu = \frac{1}{n} \sum_{i=1}^{n} x_i
$$

$$
x_i^{(c)} = x_i - \mu
$$

$$
X_c =
\begin{pmatrix}
x_1^{(c)} \\
\vdots \\
x_n^{(c)}
\end{pmatrix}
$$

入力データを列方向に中心化し、以降の射影および最適化に用いる。

### 3.2 射影と目的関数

中心化データ $X_c \in \mathbb{R}^{n \times d}$ と、単位ノルム制約 $\|v\|=1$ を満たす方向 $v \in \mathbb{R}^{d}$ を考える。まず射影を次で定義する。

$$
y = X_c v \in \mathbb{R}^{n}
$$

このとき目的関数は、射影後のデータの分散に対応する次式で与えられる。

$$
\|X_c v\|^2 = y^\top y
$$

PCA は次の最適化問題として表される。

$$
\max_{v} \|X_c v\|^2 \quad \text{ただし } \|v\| = 1
$$


実装では最小化器を用いるため、損失関数を次で定義する。

$$
\mathcal{L}(v) = -\|X_c v\|^2
$$


## 3.3 勾配最適化と正規化制約

前節で定義した損失関数

$$
\mathcal{L}(v) = -\|X_c v\|^2
$$

を勾配法により最小化する。学習対象は方向ベクトル $v \in \mathbb{R}^d$ のみであり、PyTorch の自動微分機構により勾配を計算する。

制約 $\|v\| = 1$ を満たすため、各更新ステップ後に $v$ を次のように正規化する。

$$
v \leftarrow \frac{v}{\|v\|}
$$

この正規化操作は勾配計算には含めず、単位ノルム制約を満たすための射影処理として明示的に切り離す。そのため実装では `no_grad` ブロック内で実行する。

## 4. 出力

本節では、前節で定義した各量が PyTorch による最適化PCAの出力として正しく対応していることを示す。
ここでは出力の 型・shape・対応関係のみを扱い、数値の解釈や可視化は行わない。

本実装 pca(X) は、PCA における各定義量を辞書形式で返す。
各出力は前節の数式と一対一で対応している。

{
  "mean": μ,
  "components": v,
  "scores": T,
  "eigvals": λ,
  "explained_ratio": r
}


mean            : (1, d)
components      : (d, 1)
scores          : (n, 1)
eigvals         : (1,)
explained_ratio : (1,)


対応関係は次のとおり。

- `mean`
中心化に用いた平均ベクトル

- `components`
勾配最適化により得られた主成分方向ベクトル

- `scores`
中心化データを主成分方向へ射影した値

- `eigvals`
主成分方向の分散は $\frac{1}{n}\|X_c v\|^2$ として定義する


- `explained_ratio`
全分散に対する固有値の比（寄与率）

以上により、PCA の各定義量が 最適化による実装出力として過不足なく対応していることが確認できる。

## 5.検証

本節では、事例5（NumPy・固有値分解）と事例6（PyTorch・勾配最適化）で得られた結果を比較し、両者が同一のPCA解に一致することを確認する。
比較の結果、以下の点が一致した。
- 主成分方向ベクトル（components）は、符号反転を除き一致
- 固有値（eigvals）は一致
- 寄与率（explained_ratio）は一致
- 主成分得点（scores）は、符号反転を除き一致

符号反転は、主成分方向$𝑣$と$−𝑣$ が同一の射影軸を表すことに由来するものであり、PCAの性質上の自由度である。

以上より、勾配最適化による実装が、固有値分解によるPCAと数学的にも実装的にも同一の解を与えることを確認した。

## 6. スコープ外

本事例では、可視化・解釈・複数主成分の同時最適化・既存ライブラリとの比較は扱わず、勾配最適化によるPCA実装と固有値分解PCAとの同値性確認にスコープを限定する。